# Реализация алгоритма восстановления пропущенных пробелов в тексте


## Используемые данные
Был взят список 100000 самых употребляемых русских слов с GitHub репозитория: https://github.com/hingston/russian

Этот словарь содержит лемматизированные формы слов, отсортированные по убыванию частоты употребления в русском языке.

## Алгоритм

Применяется динамическое программирование с применением вероятностной модели на основе частотности слов. Основная гипотеза: наиболее вероятное разбиение текста на слова соответствует последовательности, где сумма логарифмических вероятностей слов максимальна.

Работа алгоритма:

1. Рекурсивный перебор всех возможных разбиений
2. Морфологический анализ с помощью pymorphy3
3. Оценка вариантов по вероятностной модели
4. Мемоизация для оптимизации

In [ ]:
%pip install pymorphy3


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pymorphy3
morph = pymorphy3.MorphAnalyzer()

from collections import defaultdict, OrderedDict
import math

def load_unigrams_from_list(path, method="zipf", s=1.0, top_n=None, keep_one_letter={"и","в","с"}):
    """
    Загружает словарь слов и вычисляет логарифмические вероятности

    Параметры:
    - path: путь к файлу со словарем
    - method: метод распределения частот
    - s: параметр для Zipf-распределения
    - top_n: ограничение по количеству слов
    - keep_one_letter: односимвольные слова, которые не штрафуются
    """
    # читаем список (убираем пустые строки и пробелы)
    words = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            w = line.strip()
            if not w:
                continue
            words.append(w.lower())
    if top_n:
        words = words[:top_n]

    # создаём ranks (1..N)
    N = len(words)
    if N == 0:
        raise ValueError("Empty word list")

    freqs = []
    if method == "zipf":
        # freq_i ~ 1 / rank^s
        freqs = [1.0 / ((rank+1) ** s) for rank in range(N)]
    elif method == "linear":
        freqs = [float(N - rank) for rank in range(N)]
    elif method == "uniform":
        freqs = [1.0 for _ in range(N)]
    else:
        raise ValueError("Unknown method")


    adjusted_freqs = []
    for w, f in zip(words, freqs):
        if len(w) == 1 and w not in keep_one_letter:
            f = f / 50.0
        adjusted_freqs.append(f)

    total = sum(adjusted_freqs)
    probs = [f / total for f in adjusted_freqs]
    log_probs = {w: math.log(p) for w, p in zip(words, probs)}

    min_log = min(log_probs.values())
    unknown_log_penalty = min_log - 5.0

    return log_probs, unknown_log_penalty



from functools import lru_cache


#Разбивает текст на слова с помощью динамического программирования
def word_segmentation(text, word_probs, unknown_penalty, max_word_len=15, length_bonus_coef=1000):
    text = text.lower()

    @lru_cache(maxsize=None)
    def segment(i):
        if i == len(text):
            return 0.0, []
        best_score, best_split = float("-inf"), None
        for j in range(i + 1, min(len(text)+1, i + max_word_len + 1)):
            word = text[i:j]
            lemma = morph.parse(word)[0].normal_form
            score = word_log_probs.get(word, word_log_probs.get(lemma, unknown_log_penalty))
            rest_score, rest_split = segment(j)
            total_score = score + rest_score + length_bonus_coef * len(word)
            if total_score > best_score:
                best_score = total_score
                if j < len(text):
                    best_split = [j] + rest_split
                else:
                    best_split = rest_split
        return best_score, best_split

    return segment(0)[1]


In [ ]:
filename = "100000-russian-words.txt"
word_log_probs, unknown_log_penalty = load_unigrams_from_list(filename, method="zipf", s=1.0, top_n=200000)

In [ ]:
rows = []
with open("dataset_1937770_3.txt", "r", encoding="utf-8") as f:
    next(f)
    for line in f:
        line = line.strip()
        if not line:
            continue
        id_, text = line.split(",", 1)  # делим только по первой запятой
        rows.append((id_, text))

# создаем DataFrame
df = pd.DataFrame(rows, columns=["id", "text_no_spaces"])
print(df.head())

  id                 text_no_spaces
0  0                куплюайфон14про
1  1             ищудомвПодмосковье
2  2  сдаюквартирусмебельюитехникой
3  3     новыйдивандоставканедорого
4  4                отдамдаромкошку


In [ ]:
df

,id,text_no_spaces
0,0,куплюайфон14про
1,1,ищудомвПодмосковье
2,2,сдаюквартирусмебельюитехникой
3,3,новыйдивандоставканедорого
4,4,отдамдаромкошку
...,...,...
1000,1000,Янеусну.
1001,1001,Весна-яуженегреюпио.
1002,1002,Весна-скоровырастеттрава.
1003,1003,"Весна-выпосмотрите,каккрасиво."


In [ ]:
def predict_positions(text):
    positions = word_segmentation(text, word_log_probs, unknown_log_penalty)
    return str(positions) if positions else "[]"

In [ ]:
subset_df = df.iloc[:15]

subset_df['predicted_positions'] = subset_df['text_no_spaces'].apply(predict_positions)
print(subset_df)

    id                 text_no_spaces predicted_positions
0    0                куплюайфон14про                  []
1    1             ищудомвПодмосковье           [3, 6, 7]
2    2  сдаюквартирусмебельюитехникой        [13, 20, 21]
3    3     новыйдивандоставканедорого             [5, 20]
4    4                отдамдаромкошку                  []
5    5          работавМосквеудаленно                 [6]
6    6          куплютелевизорPhilips             [5, 14]
7    7        ищугрузчиковдляпереезда                [15]
8    8           ремонтквартирподключ                 [6]
9    9                 куплюноутбукHP                  []
10  10              ищуквартирууметро                 [3]
11  11      новаямикроволновкаSamsung             [5, 18]
12  12          срочнопродамвелосипед             [6, 12]
13  13              куплюгитаруFender                 [5]
14  14        ищурепетиторапобиологии                [15]


/tmp/ipykernel_12772/940024820.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['predicted_positions'] = subset_df['text_no_spaces'].apply(predict_positions)


In [ ]:
df['predicted_positions'] = df['text_no_spaces'].apply(predict_positions)

df.to_csv('submission.csv', index=False)
print("Файл submission.csv сохранён!")

Файл submission.csv сохранён!
